In [26]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler

In [27]:
def warn(*args, **kwargs):pass
import warnings
warnings.warn = warn

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [28]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
# Review the DataFrame

LendingDataDF = pd.read_csv("Resources/lending_data.csv")
LendingDataDF.head(10)

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
5,10100.0,7.438,50600,0.407115,4,1,20600,0
6,10300.0,7.490,51100,0.412916,4,1,21100,0
7,8800.0,6.857,45100,0.334812,3,0,15100,0
8,9300.0,7.096,47400,0.367089,3,0,17400,0
9,9700.0,7.248,48800,0.385246,4,0,18800,0


In [29]:
#Renamed Dataframe Columns To Have Better Readability
LendingDataDFRenamed = LendingDataDF.rename(columns={'loan_size': 'Loan Size', 'interest_rate': 'Interest Rate','borrower_income': 'Borrower Income', 'debt_to_income': 'Debt To Income', 'num_of_accounts': '# Of Accounts', 'derogatory_marks': 'Derogatory Marks', 'total_debt': 'Total Debt', 'loan_status': 'Loan Status'})
LendingDataDFRenamed.head(10)

,Loan Size,Interest Rate,Borrower Income,Debt To Income,# Of Accounts,Derogatory Marks,Total Debt,Loan Status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0
5,10100.0,7.438,50600,0.407115,4,1,20600,0
6,10300.0,7.490,51100,0.412916,4,1,21100,0
7,8800.0,6.857,45100,0.334812,3,0,15100,0
8,9300.0,7.096,47400,0.367089,3,0,17400,0
9,9700.0,7.248,48800,0.385246,4,0,18800,0


In [55]:
LendingDataDFRenamed.info

<bound method DataFrame.info of        Loan Size  Interest Rate  Borrower Income  Debt To Income  \
0        10700.0          7.672            52800        0.431818   
1         8400.0          6.692            43600        0.311927   
2         9000.0          6.963            46100        0.349241   
3        10700.0          7.664            52700        0.430740   
4        10800.0          7.698            53000        0.433962   
...          ...            ...              ...             ...   
77531    19100.0         11.261            86600        0.653580   
77532    17700.0         10.662            80900        0.629172   
77533    17600.0         10.595            80300        0.626401   
77534    16300.0         10.068            75300        0.601594   
77535    15600.0          9.742            72300        0.585062   

       # Of Accounts  Derogatory Marks  Total Debt  Loan Status  
0                  5                 1       22800            0  
1                  

### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [30]:
# Separate the data into labels and features
# Separate the y variable, the labels
# Separate the X variable, the features
Y = LendingDataDFRenamed['Loan Status']
X = LendingDataDFRenamed.drop(columns=['Loan Status'])

In [31]:
# Review the y variable Series
Y.head(10)

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: Loan Status, dtype: int64

In [32]:
# Review the X variable DataFrame
X.head(10)

,Loan Size,Interest Rate,Borrower Income,Debt To Income,# Of Accounts,Derogatory Marks,Total Debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000
5,10100.0,7.438,50600,0.407115,4,1,20600
6,10300.0,7.490,51100,0.412916,4,1,21100
7,8800.0,6.857,45100,0.334812,3,0,15100
8,9300.0,7.096,47400,0.367089,3,0,17400
9,9700.0,7.248,48800,0.385246,4,0,18800


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [33]:
# Check the balance of our target values
Y.value_counts()

0    75036
1     2500
Name: Loan Status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [34]:
# Import the train_test_learn module
# Split the data using train_test_split
# Assign a random_state of 1 to the function
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 1, stratify = Y)


## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [36]:
# Import the LogisticRegression module from SKLearn
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
# Fit the model using training data
Regression = LogisticRegression(random_state = 1)
RegressionModel = Regression.fit(X_train, Y_train)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [37]:
# Make a prediction using the testing data
Predictions = RegressionModel.predict(X_test)
print(Predictions)

[0 0 0 ... 0 0 0]


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [38]:
# Print the balanced_accuracy score of the model
LogisticAccuracy = balanced_accuracy_score(Y_test, Predictions)
print(LogisticAccuracy)

0.9442676901753825


In [39]:
# Generate a confusion matrix for the model
LogisticMatrix = confusion_matrix(Y_test, Predictions)
print(LogisticMatrix)

[[18679    80]
 [   67   558]]


In [40]:
# Print the classification report for the model
ClassificationReport = classification_report(Y_test, Predictions)
print(ClassificationReport)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      0.89      0.88       625

    accuracy                           0.99     19384
   macro avg       0.94      0.94      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The LRM predicted the Healthy Loan with 100% precision and the High-Risk Loan with a 87% precision while the accuracy of the overall model at 94% which is a good accuracy.

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [43]:
# Import the RandomOverSampler module form imbalanced-learn
# Instantiate the random oversampler model
# # Assign a random_state parameter of 1 to the model
# Fit the original training data to the random_oversampler model
Oversampler = RandomOverSampler(random_state=1)
X_res, Y_res = ros.fit_resample(X_train, Y_train)

In [44]:
# Count the distinct values of the resampled labels data
Y_res.value_counts()

0    56277
1    56277
Name: Loan Status, dtype: int64

### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [45]:
# Instantiate the Logistic Regression model
# Assign a random_state parameter of 1 to the model
# Fit the model using the resampled training data
# Make a prediction using the testing data
ClassificationReport2 = LogisticRegression(solver = 'lbfgs', random_state = 1)
ClassificationReport2.fit(X_res, Y_res)
Predictions2 = ClassificationReport2.predict(X_test)
pd.DataFrame({"Prediction": Predictions2, "Actual": Y_test}).reset_index(drop = True)

,Prediction,Actual
0,0,0
1,1,1
2,0,0
3,0,0
4,0,0
...,...,...
19379,0,0
19380,0,0
19381,0,0
19382,0,0


### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [47]:
# Print the balanced_accuracy score of the model 
LogisticAccuracy2 = balanced_accuracy_score(Y_test, Predictions2)
print(LogisticAccuracy2)

0.9959744975744975


In [48]:
# Generate a confusion matrix for the model
LogisticMatrix2 = confusion_matrix(Y_test, Predictions2)
print(LogisticMatrix2)

[[18668    91]
 [    2   623]]


In [49]:
# Print the classification report for the model
ClassificationReport2 = classification_report(Y_test, Predictions2)
print(ClassificationReport2)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.87      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.96     19384
weighted avg       1.00      1.00      1.00     19384



### Step 4: Answer the following question

**Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The LRM predicted the oversampled data for Healthy Loan with 100% precision and the High-Risk Loan with a 87% precision while the accuracy of the overall model at 100% which is a absolutely great accuracy to have.